
# CAE-Transformer Reduced Order Model——Kelvin–Helmholtz instability problem

## Introduction

In order to effectively reduce the design cost and cycle time of using CFD methods, the reduced-order model (ROM) has gained wide attention in recent years. For complex compressible flows, using linear methods such as Proper Orthogonal Decomposition (POD) for flow field dimensionality reduction requires a large number of modes to ensure the reconstruction accuracy. It has been shown that the modes number can be effectively reduced by using nonlinear dimensionality reduction methods. Convolutional Autoencoder (CAE) is a kind of neural network composed of encoder and decoder, which can realize data dimensionality reduction and recon-struction, and can be regarded as a nonlinear extension of POD method. CAE is used for nonlinear dimension-ality reduction, and Transformer is used for time evolution. The CAE-Transformer can obtain high reconstruction and prediction accuracy on the premise of using less latents for unsteady compressible flows.

## Framework of CAE-Transformer

The CAE-Transformer reduced order model uses a CAE network to reduce the dimensionality of the flow field, extract the characteristics of the flow data, compress it into the hidden space of the encoder, and then use the Transformer network to perform time evolution through the mechanism of self-attention on the free variables in the hidden space to obtain the free variables at other times of flow. Then, the decoder of the CAE network decodes the evolved free variables and reconstructs the flow field flow data at the corresponding time. The construction of the CAE-Transformer flow reduction model relies on the data reduction of the CAE network and the time evolution of the Transformer network. Compared with existing methods such as POD/DMD, using CAE networks for nonlinear dimensionality reduction of flow field data and Transformer networks for equation free evolution of free variables can achieve higher compression ratios and improve the efficiency of flow field prediction while ensuring the accuracy of the flow field reduction model.

+ Input：Input the flow field for a period of time.
+ Compression：Extract high-dimensional spatiotemporal flow characteristics by dimensionality reduction of the flow field using the encoder of CAE.
+ Evolution：Learning the evolution of spatiotemporal characteristics of low dimensional spatial flow fields through Transformer and predicting the next moment.
+ Reconstruction：Restore the predicted low-dimensional features of the flow field to high-dimensional space through the decoder of CAE.
+ Output：Output the predicted results of the transient flow field at the next moment.

The first step is to train the CAE network. After the training is completed, the CAE encoder is used to obtain the low dimensional features of the flow field. This low dimensional feature is used as the dataset of the Transformer network for Transformer network training.

![CAE_Transformer.png](./images/CAE_Transformer.png)

## Training environment

Import the required function library for training, where src includes dataset creation functions, network models, and training loss visualization functions.

The static GRAPH of Mindspore framework is adopted for training. Training can be done on GPU(default) or Ascend(single card).

In [1]:
import os
import time
import argparse
import numpy as np

from mindspore import nn, ops, context, save_checkpoint, set_seed, jit, data_sink
from mindflow.utils import load_yaml_config

from src import create_cae_dataset, CaeNet, plot_train_loss

In [2]:
np.random.seed(0)
set_seed(0)

In [3]:
parser = argparse.ArgumentParser(description='cae net for kh')
parser.add_argument("--mode", type=str, default="GRAPH", choices=["GRAPH", "PYNATIVE"],
                    help="Context mode, support 'GRAPH', 'PYNATIVE'")
parser.add_argument("--save_graphs", type=bool, default=False, choices=[True, False],
                    help="Whether to save intermediate compilation graphs")
parser.add_argument("--save_graphs_path", type=str, default="./graphs")
parser.add_argument("--device_target", type=str, default="GPU", choices=["GPU", "Ascend"],
                    help="The target device to run, support 'Ascend', 'GPU'")
parser.add_argument("--device_id", type=int, default=0, help="ID of the target device")
parser.add_argument("--config_file_path", type=str, default="./config.yaml")
args = parser.parse_args()

context.set_context(mode=context.GRAPH_MODE if args.mode.upper().startswith("GRAPH") else context.PYNATIVE_MODE,
                    save_graphs=args.save_graphs,
                    save_graphs_path=args.save_graphs_path,
                    device_target=args.device_target,
                    device_id=args.device_id)
use_ascend = context.get_context(attr_key='device_target') == "Ascend"

## CAE training parameter settings

Import parameter configurations for the dataset, CAE model, and optimizer from the config.yaml file.

In [4]:
config = load_yaml_config(args.config_file_path)
data_params = config["cae_data"]
model_params = config["cae_model"]
optimizer_params = config["cae_optimizer"]

The default path for saving loss files during training is optimizer_params ["summary_dir"], the weight parameters are saved in the ckpt folder.

In [5]:
summary_dir = optimizer_params["summary_dir"]
if not os.path.exists(summary_dir):
    os.mkdir(summary_dir)
ckpt_dir = os.path.join(summary_dir, 'ckpt')
if not os.path.exists(ckpt_dir):
    os.mkdir(ckpt_dir)

## Construct CAE neural network

The CAE network consists of six layers of convolution and maximum pooling to form an encoder, and seven layers of convolution and upsampling to form a decoder. Use MSELoss loss function and Adam optimizer.

In [6]:
cae = CaeNet(model_params["data_dimension"], model_params["conv_kernel_size"], model_params["maxpool_kernel_size"],
             model_params["maxpool_stride"], model_params["encoder_channels"], model_params["decoder_channels"],
             model_params["channels_dense"])

loss_fn = nn.MSELoss()
cae_opt = nn.Adam(cae.trainable_params(), optimizer_params["lr"], weight_decay=optimizer_params["weight_decay"])

if use_ascend:
    from mindspore.amp import DynamicLossScaler, auto_mixed_precision, all_finite
    loss_scaler = DynamicLossScaler(1024, 2, 100)
    auto_mixed_precision(cae, 'O1')
else:
    loss_scaler = None

## CAE dataset

Dataset download address: data_driven/cae-lstm/kh/dataset

The dataset in this case is the numerical simulation results of a two-dimensional Kelvin-Helmholtz instability problem, the instability in parallel shear flow is called KH instability. The range of coordinates x and y is \[-0.5, 0.5\], and the time t range is \[0, 1.5\]. A total of 1786 flow field snapshots, each with a matrix size of (256, 256).

After importing the dataset, perform data sinking settings.

In [7]:
cae_dataset, _ = create_cae_dataset(data_params["data_path"], data_params["batch_size"])

sink_process = data_sink(train_step, cae_dataset, sink_size=1)
train_data_size = cae_dataset.get_dataset_size()

## CAE training

Build forward_fn and train_step, start training the CAE network and visualize the training loss.

In [8]:
def forward_fn(data, label):
    logits = cae(data)
    loss = loss_fn(logits, label)
    if use_ascend:
        loss = loss_scaler.scale(loss)
    return loss

grad_fn = ops.value_and_grad(forward_fn, None, cae_opt.parameters, has_aux=False)

@jit
def train_step(data, label):
    loss, grads = grad_fn(data, label)
    if use_ascend:
        loss = loss_scaler.unscale(loss)
        if all_finite(grads):
            grads = loss_scaler.unscale(grads)
    loss = ops.depend(loss, cae_opt(grads))
    return loss

print(f"====================Start cae train=======================")
train_loss = []
for epoch in range(1, optimizer_params["epochs"] + 1):
    local_time_beg = time.time()
    cae.set_train()
    epoch_train_loss = 0
    for _ in range(train_data_size):
        epoch_train_loss = ops.squeeze(sink_process(), axis=())
    train_loss.append(epoch_train_loss)
    print(f"epoch: {epoch} train loss: {epoch_train_loss} epoch time: {time.time() - local_time_beg:.2f}s")

    if epoch % optimizer_params["save_ckpt_interval"] == 0:
        save_checkpoint(cae, f"{ckpt_dir}/cae_{epoch}.ckpt")
print(f"=====================End cae train========================")
plot_train_loss(train_loss, summary_dir, optimizer_params["epochs"], "cae")

pid:14003
====================Start cae train=======================
epoch: 1 train loss: 0.16278297 epoch time: 16.27s
epoch: 2 train loss: 0.098297514 epoch time: 2.26s
epoch: 3 train loss: 0.07453717 epoch time: 2.26s
epoch: 4 train loss: 0.05944114 epoch time: 2.27s
epoch: 5 train loss: 0.050014462 epoch time: 2.25s
...
epoch: 4396 train loss: 0.0009265681 epoch time: 2.23s
epoch: 4397 train loss: 0.00071872084 epoch time: 2.22s
epoch: 4398 train loss: 0.00075864815 epoch time: 2.22s
epoch: 4399 train loss: 0.00073658983 epoch time: 2.22
epoch: 4400 train loss: 0.0006535899 epoch time: 2.22s
====================End cae train=======================


## CAE flow field reconstruction results

After training the CAE network, run cae_prediction.py to view the training results of CAE to determine whether to continue training the LSTM network

The following figures show the real flow field, CAE flow field reconstruction results, and the error curves between them. The first two flow field results show the variation of density at different positions in the flow field over time, while the third error curve shows the average relative error of the CAE reconstructed flow field and the real flow field label over time. The error remains below 0.015 for most of the time, meeting the accuracy requirements for flow field reconstruction.

<figure class="harf">
    <img src="./images/true.gif" title="cae_train_loss" width="300"/>
    <img src="./images/cae.gif" title="cae_prediction" width="300"/>
    <img src="./images/cae_error.png" title="cae_error" width="300"/>
</center>

## Transformer training environment
Import the library required for training.

In [ ]:
import numpy as np
import mindspore as ms

from mindspore import nn, ops, context, save_checkpoint, set_seed, jit, data_sink
from mindspore.train import CheckpointConfig, ModelCheckpoint
from mindvision.engine.callback import LossMonitor

from mindflow.utils import load_yaml_config
from sympy import Q
from src import create_transformer_dataset, plot_train_loss, Informer
from cae_prediction import cae_prediction

In [ ]:
np.random.seed(42)
set_seed(42)

In [ ]:
parser = argparse.ArgumentParser(description="transformer net for KH")
parser.add_argument(
    "--mode",
    type=str,
    default="GRAPH",
    choices=["GRAPH", "PYNATIVE"],
    help="Context mode, support 'GRAPH', 'PYNATIVE'",
)
parser.add_argument(
    "--save_graphs",
    type=bool,
    default=False,
    choices=[True, False],
    help="Whether to save intermediate compilation graphs",
)
parser.add_argument("--save_graphs_path", type=str, default="./graphs")
parser.add_argument(
    "--device_target",
    type=str,
    default="GPU",
    choices=["GPU", "Ascend"],
    help="The target device to run, support 'Ascend', 'GPU'",
)
parser.add_argument(
    "--device_id", type=int, default=0, help="ID of the target device"
)
parser.add_argument("--config_file_path", type=str, default="./config.yaml")
args = parser.parse_args()

ms.set_context(device_target="GPU")
ms.set_context(mode=ms.PYNATIVE_MODE)
use_ascend = context.get_context(attr_key="device_target") == "Ascend"

## Transformer loss function Settings

CustomWithLossCell is a custom loss calculation handling loss function in MindSpore.

In [ ]:
class CustomWithLossCell(nn.Cell):
    def __init__(self, backbone, loss_fn, args):
        super(CustomWithLossCell, self).__init__(auto_prefix=False)
        self._backbone = backbone
        self._loss_fn = loss_fn
        self.args = args

    def construct(self, seq_x, seq_y):
        cast = ops.Cast()

        batch_x = cast(seq_x, ms.float32)
        batch_y = cast(seq_y, ms.float32)

        if self.args["padding"] == 0:
            dec_inp = ops.Zeros()(
                (batch_y.shape[0], self.args["pred_len"], batch_y.shape[-1]), ms.float32
            )
        else:
            dec_inp = ops.Ones()(
                (batch_y.shape[0], self.args["pred_len"], batch_y.shape[-1]), ms.float32
            )
        dec_inp = cast(
            ops.concat([batch_x[:, - self.args["label_len"] : , :], dec_inp], axis=1),
            ms.float32,
        )

        outputs = self._backbone(batch_x, dec_inp)
        batch_y = batch_y[:, -self.args["pred_len"] :, :]

        return self._loss_fn(outputs, batch_y)

## Transformer network framework and training Settings

Start by importing the Transformer model dataset setup parameters, Transformer model and optimizer parameter Settings. The default path for saving training loss is optimizer_params["summary_dir"], and the weight parameters are saved in the ckpt folder. The model is stacked with multiple encoders and decoders, using the MSELoss loss function and the Adam optimizer.

In [9]:
# prepare params
config = load_yaml_config(args.config_file_path)
data_params = config["transformer_data"]
model_params = config["transformer"]
optimizer_params = config["transformer_optimizer"]

# prepare summary file
summary_dir = optimizer_params["summary_dir"]
ckpt_dir = os.path.join(summary_dir, "ckpt")

# prepare model
model = Informer(**model_params)
loss_fn = nn.MSELoss()
optimizer = nn.Adam(
    model.trainable_params(),
    optimizer_params["lr"],
    weight_decay=optimizer_params["weight_decay"],
)

## Transformer dataset loading and processing

The Transformer network dataset is generated by CAE's encoder to create the dataset.

In [10]:
# prepare dataset
latent_true = cae_prediction(args.config_file_path)
dataset, _ = create_transformer_dataset(
    latent_true,
    data_params["batch_size"],
    data_params["time_size"],
    data_params["latent_size"],
    data_params["time_window"],
    data_params["gaussian_filter_sigma"],
)

## Transformer training

The construction of loss function network, the creation of model checkpoint callback function and the initialization of trainer begin the training of Transformer network and visualize the training loss.

In [ ]:
time_now = time.time()
loss_net = CustomWithLossCell(model, loss_fn, data_params)
config = CheckpointConfig(save_checkpoint_steps=50, keep_checkpoint_max=2)
ckpt_callback = ModelCheckpoint(
     prefix="Informer", directory=ckpt_dir, config=config
)
trainer = ms.Model(network=loss_net, optimizer=optimizer)
for epoch in range(optimizer_params["epochs"]):
    print(f">>>>>>>>>>>>>>>>>>>>Train_{epoch}<<<<<<<<<<<<<<<<<<<<")
    ts = time.time()
    trainer.train(
        1,
        dataset,
        callbacks=[
            ckpt_callback,
            LossMonitor(optimizer_params["lr"], per_print_times=50),
        ],
    )
    print("Train Time Cost:", time.time() - ts)

## Visualization of predicted flow field results

Run cae_lstm_prediction.py to view the prediction results of the CAE-LSTM reduced order model

The following figures show the actual flow field, the predicted results of the CAE-LSTM network, and the corresponding average relative error. The error of CAE-LSTM prediction results is greater than that of CAE reconstruction because the former has more LSTM evolution error than the latter, but the overall prediction time error remains below 0.02, meeting the accuracy requirements of flow field prediction.

<figure class="harf">
    <img src="./images/true2.gif" title="cae_prediction" width="300"/>
    <img src="./images/cae_lstm.gif" title="cae_lstm_prediction" width="300"/>
    <img src="./images/cae_lstm_error.png" title="cae_lstm_error" width="300"/>
</center>